<br />
<p align="center">
  <img src="../images/dtlogo.png" alt="Logo" width="111" height="100">

  <h1 align="center">Object detection</h1>
</p>



### Object Detection

Machine-learned object detection models can be extremely useful. They are faster and often more reliable than most traditional computer vision models. Aditionnally, we can use pretrained model weights to cut down immensely on training time.

Most people know about such models. They're flashy, and very easy on the eyes. Gifs like the one above are very cool, and they're always fun to watch (just realize: we tricked a machine into thinking and identifying objects and animals like a human would. That's wild!)


![gif of yolov5 in action](../images/yolov5.gif)
<p align="center">\[Source: https://analyticsindiamag.com/yolov5/\]</p>


In this exercise, you will create your own Duckietown object detection dataset. You will learn about the general structure such a dataset should follow. You will train an object detection model on said dataset. Finally, you will integrate said model into a ROS node, so that your duckiebot knows how to recognize duckie pedestrians (and thus avoid them).

### Steps:

1. Setup  
2. Investigation
3. Data collection
4. Training
5. Integration



### 1. Setup

First, we need some global variables. These allow you to change the directory where we save of of the data you will need. You can also change the image size to reflect what your final model uses, but you can worry about that later.

In [5]:
DATASET_DIR="./dataset"
IMAGE_SIZE = 416

import subprocess

def run(input, exception_on_failure=False):
    try:
        program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
    except Exception as e:
        if exception_on_failure:
            raise e
        program_output = e.output

    return program_output

def runp(input, exception_on_failure=False):
    print(run(input, exception_on_failure))
    
import contextlib
import os

@contextlib.contextmanager
def makedirs(name):
    try:
        os.makedirs(name)
    except:
        pass
    yield None

While you will build your own dataset in part 2, it would be unreasonable to ask you to build your own dataset of real images. Run the cell bellow to download a dataset of labelled real images.

In [9]:
runp(f"rm -rf {DATASET_DIR}")
runp(f"mkdir {DATASET_DIR}")
# use <!> to have a download indicator
!wget -O duckietown_object_detection_dataset.zip https://www.dropbox.com/s/bpd535fzmj1pz5w/duckietown%20object%20detection%20dataset-20201129T162330Z-001.zip?dl=0
runp(f"unzip duckietown_object_detection_dataset.zip -d {DATASET_DIR}")
runp(f"mv {DATASET_DIR}/duckietown\ object\ detection\ dataset/* {DATASET_DIR} && rm -rf {DATASET_DIR}/duckietown\ object\ detection\ dataset")
runp(f"rm -rf duckietown_object_detection_dataset.zip")



--2021-05-06 15:37:55--  https://www.dropbox.com/s/bpd535fzmj1pz5w/duckietown%20object%20detection%20dataset-20201129T162330Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.11.18, 2620:100:6050:18::a27d:b12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.11.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bpd535fzmj1pz5w/duckietown%20object%20detection%20dataset-20201129T162330Z-001.zip [following]
--2021-05-06 15:37:56--  https://www.dropbox.com/s/raw/bpd535fzmj1pz5w/duckietown%20object%20detection%20dataset-20201129T162330Z-001.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4c48da7354811da1e609b758cb.dl.dropboxusercontent.com/cd/0/inline/BN80Gg5w1b4I3j3cWtpI0H_vwqnLX0Wa3aSuO555q0nz4urX-3c5DvpRfWEIdSdGEKXAd-lUcvzwVxB16TI72buTCh4GaxqWoT3PWLHqykEFRVCALGvSvjum4_KlP8gYf0KjLHtJXjkKp18xYtLUs-bW/file# [following]
--2021-05-06 15:37:

These real images are not the right size. Run the cell bellow to resize them (and resize the associated bounding boxes accordingly).

In [11]:
import json
import os
import cv2
import numpy as np
from tqdm import tqdm

with open(f"{DATASET_DIR}/annotation/final_anns.json") as anns:
    annotations = json.load(anns)

npz_index = 0
while os.path.exists(f"{DATASET_DIR}/{npz_index}.npz"):
    npz_index += 1

def save_img(img, boxes, classes):
    global npz_index
    cv2.imwrite(f"{DATASET_DIR}/real_{npz_index}.jpg", img)
    with open(f"{DATASET_DIR}/real_{npz_index}.txt", "w") as f:
        for i in range(len(boxes)):
            f.write(f"{classes[i]} "+" ".join(map(str,boxes[i]))+"\n")
    npz_index += 1

for filename in tqdm(os.listdir(f"{DATASET_DIR}/frames")):
    img = cv2.imread(f"{DATASET_DIR}/frames/{filename}")

    orig_y, orig_x = img.shape[0], img.shape[1]
    scale_y, scale_x = IMAGE_SIZE/orig_y, IMAGE_SIZE/orig_x

    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))

    boxes = []
    classes = []

    if filename not in annotations:
        continue

    for detection in annotations[filename]:
        box = detection["bbox"]
        label = detection["cat_name"]

        if label not in ["duckie", "cone"]:
            continue

        orig_x_min, orig_y_min, orig_w, orig_h = box

        x_min = int(np.round(orig_x_min * scale_x))
        y_min = int(np.round(orig_y_min * scale_y))
        x_max = x_min + int(np.round(orig_w * scale_x))
        y_max = y_min + int(np.round(orig_h * scale_y))

        boxes.append([x_min, y_min, x_max, y_max])
        classes.append(1 if label == "duckie" else 2)

    if len(boxes) == 0:
        continue

    #make boxes to xywh format:
    def xminyminxmaxymax2xywfnormalized(box, image_size):
        xmin, ymin, xmax, ymax = np.array(box, dtype=np.float64)
        center_x = (xmin+xmax)/2
        center_y = (ymin+ymax)/2
        width = xmax-xmin
        height = ymax-ymin

        normalized = np.array([center_x, center_y, width, height])/image_size
        return np.round(normalized, 5)
    boxes = np.array([xminyminxmaxymax2xywfnormalized(box, IMAGE_SIZE) for box in boxes])
    classes = np.array(classes)-1
    
    save_img(img, boxes, classes)


100%|██████████| 1963/1963 [00:13<00:00, 146.61it/s]


You're all set! We'll explain what the code above was for later in this notebook.

### 2. Investigation

What does an object detection dataset look like? Clearly, the specifics will depend on the convention used by specific models, but the general idea is intuitive:

- We need an image
- This image might have many bounding boxes in it, so we need some sort of list of coordinates
- These bounding boxes must be associated with a class

How are the bounding boxes defined?

![image of a bounding box](../images/bbox.png)
\[Note: if you are not colorblind, you may ignore the scribbles under the colored indications for widths and heights\]

Some conventions use `x_min y_min width height`, whereas others use `x_min y_min x_max y_max`, and others use `x_center y_center width height`. In this exercise, the model we recommend ([YoloV5](https://github.com/Velythyl/yolov5)) uses `x_center y_center width height`.

And how do we actually obtain said bounding boxes? Clearly, in real-life applications, you would need to label a dataset of images by hand. But if you have access to a simulator that is able to segment images, you could obtain the bounding boxes directly from the segmented images. 

If you take a look at Pytorch's object detection [tutorial](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html), that is similar to what they do. While their images were segmented by hand, the tutorial uses the same technique that we will use here to obtain the bounding boxes. Their images look like this:

![image with bounding boxes](../images/FudanPed.png)
<p align="center">\[Source: https://www.cis.upenn.edu/~jshi/ped_html/\]</p>

And they simply calculate the min and max x and y coordinates of the segmented objects to obtain the bounding box.

We will use the segmented mode in the Duckietown simulator to compute the bounding boxes of non-segmented images.

#### What we want to detect

The goal of this exercise is to make Duckietown safer: we want to be able to detect duckie pedestrians on the road and avoid squishing them. We also want to detect trucks, busses, and cones. Here is the complete list, along with their corresponding IDs:

0. Duckie
1. Cone
2. Truck
3. Bus


### 3. Data collection


#### Format

Edit the `data_collection.py` script so that running it produces a dataset of normal simulator images and their annotations.

You will need to run the script locally, outside of a jupyter notebook and outside of Docker. The reason is simple: you will need to create and fine-tune OpenCV masks in order to filter out everything except the objects that we are interested in. Doing so requires access to your computer's rendering server.

Of course, your dataset's format depends heavily on your model. If you want to use the [YoloV5](https://github.com/Velythyl/yolov5) model that we suggest, you should colosely follow their [guide on how to train using custom data](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data).

To simplify your task, we have already written a `duckietown.yaml` file that will enable YoloV5 to understand your dataset. You will need to save your images and their bounding box data in this very specific way:

![image of dataset save format](../images/dataset_format.png)

In the parent directory (named `duckietown_dataset`), you must place two subdirectories: `train` and `val` (for now, ignore `val`, or go read "Combining with the real dataset & Validation"). Inside `train` and `val`, there must again be two subdirectories `images` and `labels`. Inside `images`, you must place your images, and inside `labels`, you must place the images' bounding box data. Notice that the label files use the same name as their corresponding image files, save for the extension. In other words, the data for `0.jpg` can be found in `0.txt`.

The format for the label files is fairly simple. For each bounding box in the corresponding image, write a row of the form `class x_center y_center width height`. Keep in mind that the pixel data must be 0-to-1 normalized (i.e., you can calculate the usual `x_center y_center width height` in pixel space and divide by the image's size). For example,

    0 0.5 0.5 0.2 0.2
    1 0.60 0.70 0.4 0.2

this file says "there is a duckie (class 0) centered in the image, whose width and height are 20% of the image's own. There is also a cone whose center is at 60% of the image's maximal x values and 70% of the image's maximal y value, and its width is 40% of the image's own while its height is 20%."

It is recommended that you read the guide posted on YoloV5's GitHub: [guide on how to train using custom data](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data).

#### Generating data

Here is some pseudocode, to get you started:

1. Take the segmented image
![image with bounding boxes](../images/normal.png)
2. For each color in the interesting colors (so the colors for duckies, trucks, busses, and cones):
    1. Remove all other colors
![image with bounding boxes](../images/normal_duckie.png)
    2. Identify each instance of the class (there might be more than one in an image. You can take inspiration from this code: https://stackoverflow.com/a/60068297/11296012
    3. Save these bounding boxes to some kind of list
3. Save the non-segmented version of the image, and write its bounding boxes+their classes to a corresponding txt file. Of coure, if you don't plan on using the YoloV5 model that we suggest, you will need to change this format accordingly.

#### Combining with the real dataset & Validation

Notice that the images that you generated here all come from the simulator. These images are only useful in simulation: real duckies, trucks, busses, and cones look fairly different! Luckily, during Step 1 of this exercise, you downloaded a dataset of real annotated images. 

You should be able to view them locally in the `real_dataset` directory.

You must now combine the real dataset with your dataset (the "simulated dataset"). But there is one extra worry: validation.

When training supervised learning models, one must worry about overfitting. You should always keep *some* of your dataset *out* of your training data. This way, you can verify that your model does not overfit to your dataset by *validating* it on the data you left out. 

During this step, while integrating the real dataset, you should worry about the validation dataset. Perhaps you should split the real dataset (80% for training data, 20% for validation? 70/30? 50/50?), and generate new simulated images for validation too!

### 4. Training

Training the model requires a GPU. If you have one, you can simply `git clone https://github.com/Velythyl/yolov5.git -b dt-obj-det` and call `python3 train.py --img 416 --batch 16 --epochs 100 --data duckietown.yaml --weights yolov5s.pt`. For reference, on a gtx1080ti and a ryzen 3700x, training took us about 20 minutes.

If you do not have a GPU, you can use google Colab. You can import the colab notebook called `dt_obj_det.ipynb`. You will also need to upload your dataset to your Google Drive. For reference, on google colab, training took us about 1h. Don't forget to change the runtime type to GPU-accelerated!

### 5. Integration

Finally, you need to integrate your model with ROS. You must edit the object detection node found in `src/object_detection/src`, and include your model and its weights in `src/object_detection/include/object_detection`. 

However, since your model will make use of TensorRT, asking it for a prediction is a bit tricky. If you go see in `exercise_ws/src/object_detection/include/object_detection`, we have provided you with some starter code. This code allows you to call your TensorRT model and have it run inference on a set of images, ultimately producing annotated images in a "output" directory. Here, we don't want to run your model on a set of images (we want to run it on a stream of images coming from your camera), and we don't want annotated images as output (as no human will look at them: only your robot will). Still, this is good starter code: you can just change the way it takes in input, and produces output. The central logic will stay the same.

